In [197]:
import folium
import pandas as pd
 
SF_COORDINATES = (37.76, -122.45)
crimedata = pd.read_csv('sfpd_incidents.csv')
 
# for speed purposes
MAX_RECORDS = 1000

In [198]:
crimedata.columns

Index(['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime', 'Row ID', 'Incident ID',
       'Incident Number', 'CAD Number', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Resolution', 'Intersection', 'CNN', 'Police District',
       'Analysis Neighborhood', 'Supervisor District', 'Latitude', 'Longitude',
       'point'],
      dtype='object')

In [199]:
crimedata['Police District'].head()

0    Ingleside
1      Mission
2     Southern
3     Southern
4      Mission
Name: Police District, dtype: object

In [219]:
crimedata.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 500 entries, 2018-01-01 02:13:00 to 2018-01-02 20:59:00
Data columns (total 27 columns):
Incident Datetime          500 non-null object
Incident Date              500 non-null object
Incident Time              500 non-null object
Incident Year              500 non-null int64
Incident Day of Week       500 non-null object
Report Datetime            500 non-null object
Row ID                     500 non-null int64
Incident ID                500 non-null int64
Incident Number            500 non-null int64
CAD Number                 453 non-null float64
Report Type Code           500 non-null object
Report Type Description    500 non-null object
Filed Online               45 non-null object
Incident Code              500 non-null int64
Incident Category          500 non-null object
Incident Subcategory       500 non-null object
Incident Description       500 non-null object
Resolution                 500 non-null object
Intersection     

In [201]:
crimedata[['Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime']].head()

Incident Date Incident Time  Incident Year Incident Day of Week  \
0    2018/01/01         01:30           2018               Monday   
1    2018/01/01         01:59           2018               Monday   
2    2018/01/01         02:28           2018               Monday   
3    2018/01/01         02:28           2018               Monday   
4    2018/01/01         02:08           2018               Monday   

          Report Datetime  
0  2018/01/01 02:13:00 AM  
1  2018/01/01 01:59:00 AM  
2  2018/01/01 02:31:00 AM  
3  2018/01/01 02:31:00 AM  
4  2018/01/01 02:08:00 AM

In [202]:
crimedata.index

RangeIndex(start=0, stop=111531, step=1)

In [203]:
crimedata.set_index(pd.to_datetime(crimedata['Report Datetime']),inplace=True)

In [204]:
crimedata.index

DatetimeIndex(['2018-01-01 02:13:00', '2018-01-01 01:59:00',
               '2018-01-01 02:31:00', '2018-01-01 02:31:00',
               '2018-01-01 02:08:00', '2018-01-01 02:08:00',
               '2018-01-01 02:08:00', '2018-01-01 02:46:00',
               '2018-01-01 02:39:00', '2018-01-01 02:29:00',
               ...
               '2018-01-02 19:40:00', '2018-01-03 17:05:00',
               '2018-01-03 19:02:00', '2018-01-03 08:34:00',
               '2018-01-03 09:03:00', '2018-01-04 09:17:00',
               '2018-01-04 14:55:00', '2018-01-04 15:00:00',
               '2018-01-04 15:54:00', '2018-01-04 15:33:00'],
              dtype='datetime64[ns]', name='Report Datetime', length=111531, freq=None)

In [205]:
crimedata = crimedata[:500]

In [254]:
import folium
import folium.plugins


m = folium.Map(location=SF_COORDINATES, zoom_start=12)
marker_cluster = folium.plugins.MarkerCluster().add_to(m)

# folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>').add_to(m)

lat = crimedata.loc['2018-01']['Latitude'].dropna().tolist()
long = crimedata.loc['2018-01']['Longitude'].dropna().tolist()
coords = zip(lat,long)

for coord in zip(lat,long):
    folium.Marker([coord[0], coord[1]]).add_to(marker_cluster)



m

KeyError: 'the label [2018-01] is not in the [index]'

In [253]:
# state_geo = os.path.join('data', 'us-states.json')
crimedata = pd.read_csv('sfpd_incidents.csv')

m2 = folium.Map(location=SF_COORDINATES, zoom_start=12)
district_geo = r'Current Police Districts.geojson'

crimedata['1'] = 1
crimedata2 = crimedata.groupby('Police District').sum()
# crimedata2

district_geo.index

crimedata2.reset_index(inplace=True)
# crimedata2.columns = ['District', 'Number']

cd = crimedata2[['Police District', '1']]
cd.columns = ['district', '1']
cd.district = cd.district.str.upper()

m2.choropleth(
    geo_data=district_geo,
    name='choropleth',
    data=cd,
    columns=['district', '1'],
    key_on='feature.properties.district',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of incidents per district'
)


folium.LayerControl().add_to(m2)

m2

C:\Users\joaquin\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
